# SCRAPPING DE PAGE-SPEED

NO ESTA BLINDADO, FALTARÍA ESO Y PASARLO A FUNCIÓN

In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as ec
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

headers = {'user_agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6.1 Safari/605.1.15'}
def INICIAR_DRIVER():
    options=Options()
    userAgent= 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6.1 Safari/605.1.15'
    options.add_argument(f'user-agent={userAgent}')
    options.add_argument('--start-maximized')
    options.add_argument('--disable-web-security')
    options.add_argument('--disable-extensions')
    options.add_argument('--disable-notifications')
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--allow-running-insecure-content')
    options.add_argument('--no-default-browser-check')
    options.add_argument('--no-first-run')
    options.add_argument('--no-proxy-server')
    options.add_argument('--disable-blink-features=AutomationControlled')
    exp_opt=['enable-automation', 'ignore-certificate-errors','enable-logging']
    options.add_experimental_option('excludeSwitches', exp_opt)
    prefs={'profile.default_content_setting_values.notifications':2, 'intl.accept_languages':['es-ES','es'],'credentials_enable_service':False}
    options.add_experimental_option('prefs',prefs)
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    return driver

driver = INICIAR_DRIVER()
wait = WebDriverWait(driver, 1.5)    
path ='https://pagespeed.web.dev'
driver.get(path)

print('Se inició el driver')

# URL DE LA PÁGINA QUE QUEREMOS SCREPEAR:

url ='https://onefootball.com/en/home'

text_input = wait.until(ec.visibility_of_element_located((By.XPATH,'//*[@id="i4"]')))
text_input.send_keys(url)
buton = wait.until(ec.element_to_be_clickable((By.XPATH,'//*[@id="yDmH0d"]/c-wiz/div[2]/div/div[2]/form/div[2]/button')))
buton.click()

print('Se entregó la URL')

time.sleep(6)

res = driver.page_source
soup = BeautifulSoup(res,'html.parser')

soups = soup.find_all('div', role='tabpanel')
columnas = ['Link','Tipo','LCP','INP','FCP','TTFB','CLS','FID']
tupla = ('Móvil','Ordenador')
link = url
df = pd.DataFrame(columns=columnas)

for i,soup in enumerate(soups):
    soup = soup.find('div', style = 'display: none;')
    metricas = soup.find('div',class_='OyzgL')
    elementos1 = metricas.find_all(class_='Ykn2A YznLle')
    elementos2 = metricas.find_all(class_='Ykn2A LR2yK')
    if i == 0:
        dic_g={'Link':url,'Tipo':tupla[i]}
        dic = {'LCP':pd.NA, 'INP': pd.NA, 'FCP':pd.NA,'TTFB':pd.NA}
        d1 = [i for i in dic]
        for i in range(len(elementos1)):
            dic[d1[i]]=elementos1[i].text
            
        dic_g.update(dic)
        
        dic2 = {'CLS':pd.NA, 'FID':pd.NA}
        d2 = [i for i in dic2]
        for j in range(len(elementos2)):
            dic2[d2[j]]=elementos2[j].text
        
        dic_g.update(dic2)
        
    else:
        dic_g={'Link':url,'Tipo':tupla[i]}
        dic = {'FCP':pd.NA, 'TTFB': pd.NA}
        d1 = [i for i in dic]
        for i in range(len(elementos1)):
            dic[d1[i]]=elementos1[i].text
            
        dic_g.update(dic)
        
        dic2 = {'LCP':pd.NA,'INP':pd.NA,'CLS':pd.NA, 'FID':pd.NA}
        d2 = [i for i in dic2]
        for j in range(len(elementos2)):
            dic2[d2[j]]=elementos2[j].text
            
        dic_g.update(dic2)
        
    df2 = pd.DataFrame([dic_g])
    df= pd.concat([df, df2], ignore_index=True)


# ALTERAR DF PARA SOLO UNA FILA (el df final es df_pivot)
df_pivot = df.pivot(index='Link', columns='Tipo', values=['LCP', 'INP', 'FCP', 'TTFB', 'CLS', 'FID'])
df_pivot.columns = [f'{metric}_{device[0].lower()}' for metric, device in df_pivot.columns]
df_pivot.reset_index(inplace=True)

print('Programa finalizado')


Se inició el driver
Se entregó la URL
Programa finalizado


In [2]:
df_pivot.head()


,Link,LCP_m,LCP_o,INP_m,INP_o,FCP_m,FCP_o,TTFB_m,TTFB_o,CLS_m,CLS_o,FID_m,FID_o
0,https://onefootball.com/en/home,3 s,2.1 s,266 ms,96 ms,2.4 s,1.8 s,1.3 s,1.1 s,0.08,0.07,29 ms,4 ms


# IDEA PROXIES


· La idea sería ver cuantas proxies públicas acepta la página, esto puede ser una medida de seguridad o algo así. 



· Hay horas que mucha gente utiliza estas proxies públicas y se saturan, haciendo que funcionen muy pocas, es    decir, la hora en que se ejecuta podría crear un sesgo en esta variable, pero solo es una propuesta.

In [3]:
import requests as res
import time
import concurrent.futures
from bs4 import BeautifulSoup

RED = "\033[31m"
GREEN = "\033[32m"
RESET = "\033[0m"
headers = {'user_agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6.1 Safari/605.1.15'}

def proxyscrape1():
    url = 'https://api.proxyscrape.com/v3/free-proxy-list/get?request=getproxies&proxy_format=protocolipport&format=json'
    r = res.get(url, headers = headers, timeout = 10)
    datos = r.json()
    http_prox = []
    for i in datos['proxies']:
        if i['protocol']=='http' and i['anonymity']=='elite':
            proxy = f"http://{i['ip']}:{i['port']}"
            http_prox.append(proxy)
    return http_prox

def proxyscrape2():
    url = 'https://free-proxy-list.net'
    r = res.get(url, headers = headers, timeout = 10)
    soup = BeautifulSoup(r.text, 'html.parser')
    filas = soup.find('tbody').find_all('tr')
    http_prox = []
    for i in filas:
        params = i.find_all('td')
        ip = params[0].text
        puerto = params[1].text
        tipo = params[4].text
        if tipo == 'elite proxy':
            proxy = f'http://{ip}:{puerto}'
            http_prox.append(proxy)
    return http_prox

def prove_prx(proxy, url):
    dic = {'proxy':proxy, 'tiempo':None,'res':None}
    try:
        proxies = {'http':proxy, 'https':proxy}
        inicio = time.time()
        r = res.get(url, headers = headers,proxies=proxies,timeout = 10)
        dic['tiempo']=time.time()-inicio
        dic['res']=r.status_code
    except:
        pass
    return dic

def bucle(proxies,url):
    yes = 0
    no = 0
    min_time = 9999
    with concurrent.futures.ThreadPoolExecutor() as exe:
        futuros = []
        for proxy in proxies:
            futuros.append(exe.submit(prove_prx,proxy,url))
            
        for futuro in concurrent.futures.as_completed(futuros):
            dic = futuro.result()
            if dic['res'] == 200:
                yes +=1
                print(f"{GREEN}Proxy {dic['proxy']} funciona --> {dic['tiempo']}s{RESET}")
                if dic['tiempo']<min_time:
                    min_time = dic['tiempo']
            else:
                print(f"{RED}Proxy {dic['proxy']} no funciona.{RESET}")
                no += 1
    if min_time == 9999:
        min_time = None
    tupla = (yes,no,min_time)
    return tupla

url = 'https://www.n3xtsports.com'
lista_prx = proxyscrape1()
lista_prx.extend(proxyscrape2())
lista_prx = list(set(lista_prx))
tupla = bucle(lista_prx,url)
print(tupla)
        

    

Proxy http://178.218.43.60:4411 no funciona.
Proxy http://185.128.107.118:4444 no funciona.
Proxy http://85.209.2.133:4444 no funciona.
Proxy http://185.105.90.92:4444 no funciona.
Proxy http://193.162.143.42:4444 no funciona.
Proxy http://193.162.143.223:4444 no funciona.
Proxy http://185.128.107.84:4444 no funciona.
Proxy http://185.105.89.120:4444 no funciona.
Proxy http://85.209.2.183:4444 no funciona.
Proxy http://45.9.75.116:4444 no funciona.
Proxy http://193.162.143.51:4444 no funciona.
Proxy http://185.128.106.37:4444 no funciona.
Proxy http://185.128.107.80:4444 no funciona.
Proxy http://85.209.2.230:4444 no funciona.
Proxy http://112.3.21.226:8060 no funciona.
Proxy http://67.43.227.227:25533 no funciona.
Proxy http://185.128.106.173:4444 no funciona.
Proxy http://45.95.203.115:4444 no funciona.
Proxy http://185.105.89.143:4444 no funciona.
Proxy http://185.105.91.159:4444 no funciona.
Proxy http://185.128.106.47:4444 no funciona.
Proxy http://85.209.153.174:5678 no funciona.